In [ ]:
%load_ext IPython.extensions.autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')
from model import FinData
# from model import train_valid_split
from model import CatboostFinModel

c:\Users\burko\OneDrive\Documents\GitHub\prices-predictions\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import datetime as dt

In [5]:
data = FinData("../../datasets/T_yandex_10min.csv")

data.insert_shifts_norms()
data.insert_rolling_means()
data.insert_exp_rolling_means()
data.insert_stochastic_oscillator()
# data.insert_high_low_diff()
# data.insert_holidays()
# data.insert_rsi()
# data.insert_bollinger()
# data.insert_random_prediction()

# data.make_binary_class_target()
data.restrict_time_down_stupidly(months=12)

In [37]:
numeric = data.numeric_features

cat = data.cat_features

args = {"iterations" : 10000, 
        "depth" : 5, 
        "learning_rate" : 0.01, # тут слегка неадекватные параметры, которые можно менять 
        "use_best_model" : True, 
        "l2_leaf_reg" : 200,
        "loss_function" : 'CrossEntropy', 
        "eval_metric" : 'Accuracy', 
        "cat_features" : cat, 
        "random_state" : 42,
        "early_stopping_rounds" : 5000}


model = CatboostFinModel(args)
model.cross_validation(data.df, cat, n_samples=5)

fist_date: 2024-09-15 04:00:00 - last_date: 2024-11-15 04:00:00
const_acc: 0.5728643216080402
fist_date: 2024-08-14 04:00:00 - last_date: 2024-10-14 04:00:00
const_acc: 0.5742574257425742
fist_date: 2024-09-13 04:00:00 - last_date: 2024-11-13 04:00:00
const_acc: 0.6030150753768844
fist_date: 2024-08-04 07:00:00 - last_date: 2024-10-04 07:00:00
const_acc: 0.542713567839196
(199,)
On trial 4 with date 2024-08-04 00:00:00 got accuracy 0.592964824120603
fist_date: 2024-08-04 07:00:00 - last_date: 2024-10-04 07:00:00
const_acc: 0.542713567839196
fist_date: 2024-08-16 04:00:00 - last_date: 2024-10-16 04:00:00
const_acc: 0.4723618090452261
fist_date: 2024-08-24 04:00:00 - last_date: 2024-10-24 04:00:00
const_acc: 0.5276381909547738
fist_date: 2024-08-16 04:00:00 - last_date: 2024-10-16 04:00:00
const_acc: 0.4723618090452261
(199,)
On trial 3 with date 2024-08-16 00:00:00 got accuracy 0.5728643216080402
fist_date: 2024-08-04 07:00:00 - last_date: 2024-10-04 07:00:00
const_acc: 0.54271356783919

0.603715092877322

In [68]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

model = CatBoostClassifier(
    iterations=500, depth=5,use_best_model=True, loss_function='CrossEntropy', eval_metric='Accuracy', verbose=False)

encoder = ColumnTransformer(transformers=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'),  data.cat_features)
], remainder='passthrough')

pipeline = Pipeline([
    ('encoder', encoder),
    ('classifier', CatBoostClassifier())
])

X = data.df.drop(columns=["direction_binary", 'utc'])
y = data.df["direction_binary"]

rfe = RFE(pipeline.steps[1][1], n_features_to_select=5, step=1)
X_rfe = rfe.fit_transform(X.values, y.values)

scores = cross_val_score(pipeline, X_rfe, y, scoring='accuracy', cv=5)

print("Точность модели после RFE (n_features_to_select = %d):" % n_features_to_select)
print("Mean Accuracy: %.3f" % scores.mean())


print("Выбранные признаки:", rfe.support_)
print("Индексы выбранных признаков:", rfe.ranking_)


Learning rate set to 0.04222
0:	learn: 0.6909709	total: 15.6ms	remaining: 15.5s
1:	learn: 0.6887917	total: 31.1ms	remaining: 15.5s
2:	learn: 0.6868504	total: 46.9ms	remaining: 15.6s
3:	learn: 0.6849438	total: 61.3ms	remaining: 15.3s
4:	learn: 0.6832630	total: 73.1ms	remaining: 14.5s
5:	learn: 0.6816380	total: 86ms	remaining: 14.2s
6:	learn: 0.6800822	total: 98.7ms	remaining: 14s
7:	learn: 0.6786023	total: 111ms	remaining: 13.8s
8:	learn: 0.6772539	total: 124ms	remaining: 13.6s
9:	learn: 0.6760871	total: 136ms	remaining: 13.4s
10:	learn: 0.6749553	total: 149ms	remaining: 13.4s
11:	learn: 0.6739937	total: 161ms	remaining: 13.2s
12:	learn: 0.6730579	total: 173ms	remaining: 13.1s
13:	learn: 0.6721235	total: 184ms	remaining: 13s
14:	learn: 0.6712626	total: 197ms	remaining: 12.9s
15:	learn: 0.6705710	total: 209ms	remaining: 12.8s
16:	learn: 0.6698203	total: 225ms	remaining: 13s
17:	learn: 0.6692195	total: 238ms	remaining: 13s
18:	learn: 0.6686301	total: 251ms	remaining: 12.9s
19:	learn: 0.66